In [ ]:
%matplotlib inline

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import cv, os, glob

import pandas as pd
import numpy as np
from skimage import io
from PIL import Image
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

import matplotlib.pyplot as plt

## Task 1. Dataset
First, the data is prepared. To this end, all the training/test/validation files are loaded into dataframes.

In [ ]:
train_dir = './ds/train/'
test_dir = './ds/test/'

classes = [os.path.basename(x[0]) for x in os.walk("./ds/train") if x[0]][1:]
num_classes = len(classes)
print(classes)

In [ ]:
def load_data(path):
    data = []
    for c, category in enumerate(classes):
        images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(path + category) for f in filenames]

        for img_path in images:
            # Note that we apply the same preprocessing and target images of size 224x224
            # in order to match the setup of original VGG16 model
            img = image.load_img(img_path, target_size=(224, 224))  
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x) 

            data.append({'x' : np.array(x[0]), 'y' : c})
            # data.append({'x' : np.array(img), 'y' : c})

    return data


data_train_val = load_data(train_dir)
data_test = load_data(test_dir)

In [ ]:
# Set up all arrays for train, test, validation
X_train_val, y_train_val = np.array([t["x"] for t in data_train_val]), [t["y"] for t in data_train_val]
X_test, y_test = np.array([t["x"] for t in data_test]), [t["y"] for t in data_test]

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=42, test_size=0.2)

# Convert classes to one-hot vectors
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

print('Train images:', len(y_train), y_train.shape)
print('Test images:', len(y_test), y_test.shape)
print('Val images:', len(y_val), y_val.shape)

## Task 2. Data analysis and initial model

In [ ]:
unique, counts = np.unique(y_train_val, return_counts=True)
ticks = np.arange(len(counts))
plt.figure(figsize=(14,6))
plt.grid(color='w', axis='y')
plt.bar(ticks, counts)
plt.xticks(ticks, classes, rotation=75)
plt.title('Number of images per class')
plt.show()

## v2

In [ ]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
inp = vgg.input

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(num_classes, activation='softmax')

# connect our new layer to the second to last layer in VGG, and make a reference to it
out = new_classification_layer(vgg.layers[-2].output)

# create a new network between inp and out
model_new = keras.models.Model(inp, out)

In [ ]:
# make all layers untrainable by freezing weights (except for last layer)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

In [ ]:
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history2 = model_new.fit(X_train, y_train, 
                         batch_size=128,
                         epochs=10,
                         validation_data=(X_val, y_val),
                         callbacks=callbacks_list)

In [ ]:
loss, accuracy = model_new.evaluate(X_test, y_test)

print('Test loss:', loss)
print('Test accuracy:', accuracy)